# Assessment of the SST climatology and variability

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import tqdm
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time
# Note: Time period from December year_start-1 to November year_stop
year_start = 1982
year_stop = 2011

## Define Parameters

In [ ]:
# Requests
request_dicts = {
    "ESACCI": {
        "collection_id": "satellite-sea-surface-temperature",
        "request": {
            "processinglevel": "level_4",
            "format": "zip",
            "variable": "all",
            "sensor_on_satellite": "combined_product",
            "version": "2_1",
        },
        "chunks": {"year": 1, "month": 1},
    },
    "GMPE": {
        "collection_id": "satellite-sea-surface-temperature-ensemble-product",
        "request": {
            "format": "zip",
            "variable": "all",
        },
        "chunks": {"year": 1, "month": 1, "day": 12},  # CDS limit is 12
    },
}

# Parameters to speed up I/O
open_mfdataset_kwargs = {
    "concat_dim": "time",
    "combine": "nested",
    "data_vars": "minimal",
    "coords": "minimal",
    "compat": "override",
    "parallel": True,
}

## Functions to cache

In [ ]:
def get_masked_sst(ds):
    da = ds["analysed_sst"]
    if "mask" in ds:
        da = da.where(ds["mask"] == 1)
    with xr.set_options(keep_attrs=True):
        da -= 273.15
    da.attrs["units"] = "°C"
    return da


def add_chunksizes(da):
    da.encoding["chunksizes"] = tuple(map(max, da.chunks))
    return da


def rechunk(obj):
    """Use NetCDF chunks."""
    chunks = {"time": 1, "year": 1, "season": 1, "latitude": 1_200, "longitude": 2_400}
    obj = obj.chunk(
        **{dim: chunksize for dim, chunksize in chunks.items() if dim in obj.dims}
    )
    if isinstance(obj, xr.DataArray):
        return add_chunksizes(obj)
    for da in obj.data_vars.values():
        add_chunksizes(da)
    return obj


def compute_time_reductions(ds, func, **kwargs):
    ds = rechunk(ds)
    da = get_masked_sst(ds)
    da_reduced = rechunk(func(da, **kwargs))
    if "season" in da_reduced.dims:
        da_reduced = da_reduced.sel(season=sorted(set(da["time"].dt.season.values)))
    return rechunk(da_reduced).to_dataset()

## Download and transform

In [ ]:
# Settings
reductions = ("mean", "std")
season_month_dict = {
    "DJF": {"12", "01", "02"},
    "MAM": {"03", "04", "05"},
    "JJA": {"06", "07", "08"},
    "SON": {"09", "10", "11"},
}
start = f"{year_start-1}-12"
stop = f"{year_stop}-11"

# Initialize variables
datasets_annual = {}
datasets_seasonal = {}
for product, request_dict in request_dicts.items():
    kwargs = {
        "collection_id": request_dict["collection_id"],
        "transform_chunks": False,
        "chunks": request_dict["chunks"],
        **open_mfdataset_kwargs,
    }

    # Annual
    print(f"{product=}")
    annual_ds_list = []
    for year in tqdm.tqdm(range(year_start, year_stop + 1), desc="annual"):
        # Note: year from December year-1 to November year
        annual_requests = download.update_request_date(
            request_dict["request"],
            start=f"{year-1}-12",
            stop=f"{year}-11",
            stringify_dates=True,
        )
        tmp_ds_list = []
        for reduction in reductions:
            func = getattr(diagnostics, f"time_weighted_{reduction}")
            ds = download.download_and_transform(
                **kwargs,
                requests=annual_requests,
                transform_func=compute_time_reductions,
                transform_func_kwargs={"func": func, "weights": False},
            )
            tmp_ds_list.append(rechunk(ds).expand_dims(reduction=[reduction]))
        annual_ds_list.append(
            xr.concat(tmp_ds_list, "reduction").expand_dims(year=[year])
        )
    datasets_annual[product] = xr.concat(annual_ds_list, "year")

    # Seasonal
    requests = download.update_request_date(
        request_dict["request"],
        start=start,
        stop=stop,
        stringify_dates=True,
    )
    seasonal_ds_list = []
    for season, months in tqdm.tqdm(season_month_dict.items(), desc="season"):
        season_requests = [
            {
                k: v
                if k != "month"
                else sorted(set({v} if isinstance(v, str) else v) & months)
                for k, v in r.items()
            }
            for r in requests
        ]
        tmp_ds_list = []
        for reduction in reductions:
            # Seasonal
            func = getattr(diagnostics, f"seasonal_weighted_{reduction}")
            ds = download.download_and_transform(
                **kwargs,
                requests=season_requests,
                transform_func=compute_time_reductions,
                transform_func_kwargs={"func": func, "weights": False},
            )
            tmp_ds_list.append(rechunk(ds).expand_dims(reduction=[reduction]))
        seasonal_ds_list.append(xr.concat(tmp_ds_list, "reduction"))
    datasets_seasonal[product] = xr.concat(seasonal_ds_list, "season")

del tmp_ds_list, annual_ds_list, seasonal_ds_list

## Plot low resolution seasonal maps

In [ ]:
datasets = []
for product, ds in datasets_seasonal.items():
    if product == "ESACCI":
        ds = ds.coarsen(latitude=5, longitude=5).mean()
        ds["latitude"] = ds["latitude"].round(3)
        ds["longitude"] = ds["longitude"].round(3)
    datasets.append(ds.expand_dims(product=[product]))
ds = xr.concat(datasets, "product")

for reduction, da in ds["analysed_sst"].groupby("reduction"):
    title = f"{reduction.title()} ({start}, {stop})"
    kwargs = {"projection": ccrs.Robinson(), "plot_func": "contourf"}

    # Reduction
    plot.projected_map(
        da,
        cmap="Spectral_r" if reduction == "mean" else "tab20b",
        center=False,
        col="product",
        row="season",
        **kwargs,
    )
    plt.suptitle(title)
    plt.show()

    # Bias
    with xr.set_options(keep_attrs=True):
        bias = da.diff("product").squeeze()
    plot.projected_map(
        bias, cmap="PRGn", center=True, col="season", col_wrap=2, **kwargs
    )
    plt.suptitle("Bias of " + title + "\n" + " - ".join(da["product"][::-1].values))
    plt.show()